# Module 10 - Programming Assignment

## Directions

1. Change the name of this file to be your JHED id as in `jsmith299.ipynb`. Because sure you use your JHED ID (it's made out of your name and not your student id which is just letters and numbers).
2. Make sure the notebook you submit is cleanly and fully executed. I do not grade unexecuted notebooks.
3. Submit your notebook back in Blackboard where you downloaded this file.

*Provide the output **exactly** as requested*

# Forward Planner

## Unify

Use the accompanying `unification.py` file for unification. For this assignment, you're almost certainly going to want to be able to:

1. specify the problem in terms of S-expressions.
2. parse them.
3. work with the parsed versions.

`parse` and `unification` work exactly like the programming assignment for last time.

In [1]:
from unification import parse, unification
from pprint import pprint
from copy import deepcopy

## Forward Planner

In this assigment, you're going to implement a Forward Planner. What does that mean? If you look in your book, you will not find pseudocode for a forward planner. It just says "use state space search" but this is less than helpful and it's a bit more complicated than that. **(but please please do not try to implement STRIPS or GraphPlan...that is wrong).**

At a high level, a forward planner takes the current state of the world $S_0$ and attempts to derive a plan, basically by Depth First Search. We have all the ingredients we said we would need in Module 1: states, actions, a transition function and a goal test. We have a set of predicates that describe a state (and therefore all possible states), we have actions and we have, at least, an implicit transition function: applying an action in a state causes the state to change as described by the add and delete lists.

Let's say we have a drill that's an item, two places such as home and store, and we know that I'm at home and the drill is at the store and I want to go buy a drill (have it be at home). We might represent that as:

<code>
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)",
    "(at Money Bank)"
]
</code>

And we have a goal state:

<code>
goal = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)",
    "(at Money Bank)"
]
</code>

The actions/operators are:

<code>
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}
</code>

These will all need to be parsed from s-expressions to the underlying Python representation before you can use them. You might as well do it at the start of your algorithm, once. The order of the conditions is *not* arbitrary. It is much, much better for the unification and backtracking if you have the "type" predicates (item, place, agent) before the more complex ones. Trust me on this.

As for the algorithm itself, there is going to be an *outer* level of search and an *inner* level of search.

The *outer* level of search that is exactly what I describe here: you have a state, you generate successor states by applying actions to the current state, you examine those successor states as we did at the first week of the semester and if one is the goal you stop, if you see a repeat state, you put it on the explored list (you should implement graph search not tree search). What could be simpler?

It turns out the Devil is in the details. There is an *inner* level of search hidden in "you generate successor states by applying actions to the current state". Where?

How do you know if an action applies in a state? Only if the preconditions successfully unify with the current state. That seems easy enough...you check each predicate in the conditions to see if it unifies with the current state and if it does, you use the substitution list on the action, the add and delete lists and create the successor state based on them.

Except for one small problem...there may be more than one way to unify an action with the current state. You must essentially search for all successful unifications of the candidate action and the current state. This is where my question through the semester appliesm, "how would you modify state space search to return all the paths to the goal?"

Unification can be seen as state space search by trying to unify the first precondition with the current state, progressively working your way through the precondition list. If you fail at any point, you may need to backtrack because there might have been another unification of that predicate that would succeed. Similarly, as already mentioned, there may be more than one.

So...by using unification and a properly defined <code>successors</code> function, you should be able to apply graph based search to the problem and return a "path" through the states from the initial state to the goal. You'll definitely want to use graph-based search since <code>( drive Me Store), (drive Me Home), (drive Me Store), (drive Me Home), (drive Me Store), (buy Me Store Drill), (drive Me Home)</code> is a valid plan.

Your function should return the plan...a list of actions, fully instantiated, for the agent to do in order: [a1, a2, a3]. If you pass an extra intermediate=True parameter, it should also return the resulting state of each action: [s0, a1, s1, a2, s2, a3, s3].

-----

(you can just overwrite that one and add as many others as you need). Remember to follow the **Guidelines**.


-----

So you need to implement `forward_planner` as described above. `start_state`, `goal` and `actions` should all have the layout above and be s-expressions.

Your implementation should return the plan as a **List of instantiated actions**. If `debug=True`, you should print out the intermediate states of the plan as well.

**parse_set**<br>
The `parse_set` parses the data in the dictionary

Parameters:
* **dct** dictionary we want to parse the data of

The function return the dictionary with parsed data.</br>
For example if the dct is as shown below
```
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "delete": [
            "(at ?agent ?from)"
        ]
    },
```
retuns:<br>
```actions = 
    {
    "drive": ["drive", "?agent", "?from", "?to"]
    "delete": ["at", "?agent", "?from"]
    }
```

In [2]:
def parse_set(dct):
    for key in dct:
        if type(dct[key]) == str:
            dct[key] = parse(dct[key])
        elif type(dct[key]) == list:
                dct[key] = parse(dct[key][0]) if len(dct[key]) == 1 else [parse(item) for item in dct[key]]
        elif type(dct[key]) == dict:
            dct[key] = parse_set(dct[key])
    return dct

**is_terminal**<br>
The `is_terminal` check if the current_state and the goal are equal.

Parameters:
* **current_state** is the info that we need to match to the goal state
* **goal** is the info about the goal state

The function returns `True` if current_state and goal_match otherwise it returns `False`</br>
For example if the current_state and goal are as shown below
```
current_stae=[
    "(item Saw)",
    "(item Drill)",
    "(place Home)",]
    
goal=[
    "(item Saw)",
    "(item Drill)",
    "(place Home)",] 
```
retuns:<br>
`True`

In [3]:
def is_terminal(current_state, goal):   # are current_state and goal_states the same?
    current_check = [1 for ind in range(len(current_state)) if current_state[ind] in goal]
    goal_check = [1 for ind in range(len(goal)) if goal[ind] in current_state]
    if len(current_check) == len(current_state) and len(goal_check) == len(goal):
        return True
    return False

**apply_result**<br>
The `apply_result` applies the substitutions in the expr_dct

Parameters:
* **result** is dictionary contaning the substitution that need to be made
* **expr_dct** is the dictionary we want to make the substitutions to.

The function does not return anything</br>
For example if the result and expr_dct are as shown below
```
result= {?agent: Me }
expr_dct={
    "drive": {
    "drive": ["drive", "?agent", "?from", "?to"]
    "delete": ["at", "?agent", "?from"]
    }
```
retuns:<br>
it modified expr_dct to the following
```
expr_dct={
    "drive": {
    "action": ["drive", "Me", "?from", "?to"]
    "delete": ["at", "Me", "?from"]
    }
```

In [4]:
def apply_result(result, expr_dct):
    read_from = enumerate(expr_dct) if type(expr_dct) == list else expr_dct.items()   # dictionary or list?
    for indx, value in read_from:
        if type(value) == list:
            apply_result(result, value)
        else:
            if value in result:
                expr_dct[indx] = result[value]

**update_current_state**<br>
The `update_current_state` updates the current_state given the actions

Parameters:
* **current_state** is state we want to update
* **acton** is the action we want to use to update current state

The function returns `dictionary` containing action and updated_current_state </br>
For example if the current_state and action are as shown below
```
current_stae=[
    [item, Saw],
    [item, Drill],
    [place, Home],
    [at, me, Store]]

action = {
        "action": [drive, Me, Store, Home],
        "conditions": [
            [agent, Me],
            [at, Me, Store]
        ],
        "add": [
            [at, Me, Home]
        ],
        "delete": [
            [at, Me, Store]
        ]
    }

```
retuns:<br>
it returns updated state
```
updated_state={
      "action": [drive, Me, Store, Home]
      "current_state": [[item, Saw],
                        [item, Drill],
                        [place, Home],
                        [at, me, Home]]}
```

In [5]:
def update_current_state(current_state, action):
    update_state = deepcopy(current_state)
    action_state = deepcopy(action)
    
    tmp_cond = "("+" ".join(action_state["add"])+")"  # add condition to updated state
    if tmp_cond not in update_state:
        update_state += [tmp_cond]
    
    tmp_cond = "("+" ".join(action_state["delete"])+")" # remove delete condition
    if tmp_cond in update_state:
        update_state.remove(tmp_cond)
    
    del action_state["add"]
    del action_state["delete"]
    del action_state["conditions"]
    action_state["current_state"] = update_state  # save update state in action dictionary
    return action_state

**successor_helper**<br>
The `successor_helper` is a helper function for `successor` which recursively updates the current_state given the actions.

Parameters:
* **current_state** lis the list containing info we want to determine if they meet the action condition.
* **action** dictionary contains information like conditions, add, delete, action
* **substitutes** is dictionary used to keep track of all the substitutes

returns<br>
The function does not return anything, but it does modify action dictionary. </br>

In [6]:
def successor_helper(current_state, action, substitutes):
    for action_cond in action["conditions"]:
        for c_cond in current_state:
            result = unification(action_cond, parse(c_cond))
            if result != False:
                action["conditions"].remove(action_cond)
                substitutes.update(result)
                apply_result(result, action) if result != {} else None
                if action["conditions"] != []:
                    successor_helper(current_state, action, substitutes)
                break

**successor**<br>
The `successor` function finds all successors of the given current state. 

Parameters:
* **current_state** is the current state we want to determine the successors for
* **action** dictionary contains information like conditions, add, delete, action about different actions

returns<br>
The function returns a `list of dictionaries`. where each dictionary contains informatioon about a successor state and its updated current state.

In [7]:
def successor(current_state, actions):
    ac_parsed, succ_lst = deepcopy(actions), []
    for action in ac_parsed:
        for action_cond in ac_parsed[action]["conditions"]:
            for c_cond in current_state:
                ac_copy, substitutes = deepcopy(ac_parsed), {}
                result = unification(action_cond, parse(c_cond))
                if result != False:
                    ac_copy[action]["conditions"].remove(action_cond)  # current condition matched so remove
                    substitutes= result
                    apply_result(result, ac_copy[action])
                    successor_helper(current_state, ac_copy[action], substitutes) 
                    if ac_copy[action]["conditions"] == []:
                        check = [0 for dct in succ_lst if " ".join(dct["action"]) == " ".join(ac_copy[action]["action"])]  # additiona check to make sure we don't have dupes
                        if len(check) == 0:
                            new_current_state = update_current_state(current_state, ac_copy[action])
                            new_current_state["subs"] = substitutes
                            succ_lst += [new_current_state]
    return succ_lst

**in_check**<br>
The `in_check` is a helper function for `forward planner`. It cheks if the given action is in the explored and frontier.

Parameters:
* **action** the action we want to find if it is in the explored or frontier
* **explored** is a list contaning explored actions
* **frontier** is a stack of dictionaries contaning actions and its current state

The function returns `True` if action is not in explored and in frontier otherwise it returns `False`
For example if the explored and frontier as given below and action  = "buy Me Store Drill"
```
explored = [drive Me Home Bank, drive Me Bank Home, drive Me Home Store]
frontier = {action:buy Me Store Saw, current state: []},
           {action:drive Me Store Home, current_state = []}
```
It returns<br>
`True`

In [8]:
def in_check(action,explored, frontier):
    explored_check, frontier_check = True, True
    if " ".join(action) in explored:
        explored_check = False  # explored check failed
    for dct in frontier:
        if " ".join(dct["action"]) == action:
            frontier_check = False  # frontier check failed
            break
    return explored_check & frontier_check

**backtrack**<br>
The `backtrack` is a helper function for `forward planner`. It takes tha path dictionary and backtracks the the path to the initial state given the last_action.

Parameters:
* **path** THe dictionary containing the parent action of each of the actions.
* **last_action** the action whose current state matched the goal.

The function returns `list of actions` which is the final path.
For example if the path is as given below and last action was "drive me Bank Home"
```
path = {'drive Me Home Bank': 'start action',          # in format node:parent_node
        'drive Me Bank Home': 'drive Me Home Bank'}
```
It returns<br>
`[drive Me Home Bank, 'drive Me Bank Home]`

In [9]:
def backtrack(path, last_action):
    action = last_action
    final_path = [action]
    while path[action] != "start action":
        action = path[action]
        final_path.insert(0, action)
    return final_path

**forward_planner**<br>
The `forward_planner` implements a forward planner. It finds the actions that can lead from the current state to the goal.

Parameters:
* **start_state** is the state we will start the algorithm with
* **goal** is the state we want to reach.
* **acions** is dictionary of possible actions.
* **debug** is flag to indicate if we want to print debug statements. It is set to False by defauly.

returns<br>
The function returns `list of actions` which is the final path.

In [10]:
def forward_planner( start_state, goal, actions, debug=False):
    path = {}
    start = {"action":["start","action"],"subs": {}, "current_state" : start_state}
    frontier, explored = [start], []
    actns = parse_set(deepcopy(actions))
    while frontier != []:
        current_state = frontier.pop()
        print(f"Action: {' '.join(current_state['action'])}") if debug else None
        if is_terminal(current_state["current_state"], goal):
            print(current_state["current_state"])
            return backtrack(path, " ".join(current_state["action"]))                # backtrack the path
        children = successor(current_state["current_state"], actns)
        for child in children:
            if in_check(child["action"], explored, frontier):                        # check if action is in explored or frontier 
                path[" ".join(child["action"])] = " ".join(current_state["action"])  # update the path with its parent node
                frontier += [child]
        explored += [" ".join(current_state["action"])]
    return []

You will be solving the problem from above. Here is the start state:

In [11]:
start_state = [
    "(item Saw)",
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",
    "(agent Me)",
    "(at Me Home)",
    "(at Saw Store)",
    "(at Drill Store)"
]

The goal state:

In [12]:
goal = [
    "(item Saw)",    
    "(item Drill)",
    "(place Home)",
    "(place Store)",
    "(place Bank)",    
    "(agent Me)",
    "(at Me Home)",
    "(at Drill Me)",
    "(at Saw Store)"    
]

and the actions/operators:

In [13]:
actions = {
    "drive": {
        "action": "(drive ?agent ?from ?to)",
        "conditions": [
            "(agent ?agent)",
            "(place ?from)",
            "(place ?to)",
            "(at ?agent ?from)"
        ],
        "add": [
            "(at ?agent ?to)"
        ],
        "delete": [
            "(at ?agent ?from)"
        ]
    },
    "buy": {
        "action": "(buy ?purchaser ?seller ?item)",
        "conditions": [
            "(item ?item)",
            "(place ?seller)",
            "(agent ?purchaser)",
            "(at ?item ?seller)",
            "(at ?purchaser ?seller)"
        ],
        "add": [
            "(at ?item ?purchaser)"
        ],
        "delete": [
            "(at ?item ?seller)"
        ]
    }
}

**Note** The facts for each state are really an ordered set. When comparing two states, you may need to convert them to a Set first.

In [14]:
plan = forward_planner( start_state, goal, actions, debug = True)

Action: start action
Action: drive Me Home Bank
Action: drive Me Bank Home
Action: drive Me Home Store
Action: buy Me Store Drill
Action: buy Me Store Saw
Action: drive Me Store Home
Action: drive Me Home Home
Action: drive Me Store Home
['(item Saw)', '(item Drill)', '(place Home)', '(place Store)', '(place Bank)', '(agent Me)', '(at Saw Store)', '(at Drill Me)', '(at Me Home)']


In [15]:
for el in plan:
    print(el)

drive Me Home Bank
drive Me Bank Home
drive Me Home Store
buy Me Store Drill
buy Me Store Saw
drive Me Store Home


## Before You Submit...

1. Did you provide output exactly as requested?
2. Did you re-execute the entire notebook? ("Restart Kernel and Rull All Cells...")
3. If you did not complete the assignment or had difficulty please explain what gave you the most difficulty in the Markdown cell below.
4. Did you change the name of the file to `jhed_id.ipynb`?

Do not submit any other files.